<a href="https://colab.research.google.com/github/khatrikajal/Backend/blob/main/finalbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!mkdir -p my_legal_chatbot
!touch my_legal_chatbot/__init__.py
!touch my_legal_chatbot/legal_bot.py
!touch my_legal_chatbot/appointment_manager.py
!touch my_legal_chatbot/main.py
!touch my_legal_chatbot/requirements.txt

In [ ]:
!zip -r my_legal_chatbot.zip my_legal_chatbot/




	zip warning: name not matched: my_legal_chatbot/

zip error: Nothing to do! (try: zip -r my_legal_chatbot.zip . -i my_legal_chatbot/)


In [ ]:
%cd /content/my_legal_chatbot
!pip install -r requirements.txt


/content/my_legal_chatbot


In [ ]:
# Move into the correct folder
%cd /content/my_legal_chatbot

# Run the app
!python3 main.py


/content/my_legal_chatbot
2025-03-02 09:48:19.534864: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740908899.560396   33256 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740908899.568042   33256 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-02 09:48:19.593798: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://d8db28a43dfbb61c5c.

In [ ]:
# main.py

import gradio as gr
from appointment_manager import handle_appointment, view_appointment, delete_appointment
from legal_bot import get_legal_answer

# Function to show correct placeholder based on user's choice
def update_placeholder(choice):
    if choice == "Book Appointment":
        return gr.update(placeholder="Enter information like: name,phone,date(YYYY-MM-DD),time(HH:MM AM/PM)")
    elif choice == "Check Appointment":
        return gr.update(placeholder="Enter your phone number to check appointment")
    elif choice == "Delete Appointment":
        return gr.update(placeholder="Enter your phone number to delete appointment")
    elif choice == "Ask Legal Question":
        return gr.update(placeholder="Ask your legal question (e.g., What is Article 23?)")
    return gr.update(placeholder="Type your message...")

# Main chatbot logic to handle user input based on choice
def chatbot_interface(message, history, choice):
    if choice == "Book Appointment":
        response = handle_appointment(message)
        history.append({"role": "user", "content": message})
        history.append({"role": "assistant", "content": response})
        return "", history

    elif choice == "Check Appointment":
        phone = message.strip()
        response = view_appointment(phone)
        history.append({"role": "user", "content": f"Check appointment for {phone}"})
        history.append({"role": "assistant", "content": response})
        return "", history

    elif choice == "Delete Appointment":
        phone = message.strip()
        response = delete_appointment(phone)
        history.append({"role": "user", "content": f"Delete appointment for {phone}"})
        history.append({"role": "assistant", "content": response})
        return "", history

    elif choice == "Ask Legal Question":
        response = get_legal_answer(message)
        history.append({"role": "user", "content": message})
        history.append({"role": "assistant", "content": response})
        return "", history

with gr.Blocks() as demo:
    gr.Markdown("## ⚖️ Legal & Appointment Assistant")

    chatbot = gr.Chatbot([], label="Chat", type="messages")

    choice = gr.Radio(
        ["Book Appointment", "Check Appointment", "Delete Appointment", "Ask Legal Question"],
        label="Choose an option"
    )

    message = gr.Textbox(placeholder="Type your message...")

    submit = gr.Button("Submit")

    # Dynamically update placeholder text based on selected option
    choice.change(fn=update_placeholder, inputs=choice, outputs=message)

    submit.click(chatbot_interface, inputs=[message, chatbot, choice], outputs=[message, chatbot])

if __name__ == "__main__":
    demo.launch(share=True)


KeyboardInterrupt: 

In [ ]:
!curl -u 'khatrikajal' https://api.github.com/user/repos -d '{"name":"your_repo_name"}'


Enter host password for user 'khatrikajal':
{
  "message": "Requires authentication",
  "documentation_url": "https://docs.github.com/rest/repos/repos#create-a-repository-for-the-authenticated-user",
  "status": "401"
}


In [16]:
!git config --global init.defaultBranch main


In [ ]:
!git config --global user.name "khatrikajal"
!git config --global user.email "kajalkhatri9373@gmail.com"


In [17]:
%cd /content/my_legal_chatbot
!git init


/content/my_legal_chatbot
Reinitialized existing Git repository in /content/my_legal_chatbot/.git/


In [18]:
cd /content/my_legal_chatbot


/content/my_legal_chatbot


In [19]:
!git add .



In [20]:
!git commit -m "initial commit"
!git push origin main


On branch master
nothing to commit, working tree clean
error: src refspec main does not match any
error: failed to push some refs to 'origin'


In [11]:
!mv /content/my_legal_chatbot/constitution_qa.json/* /content/my_legal_chatbot/
!mv /content/my_legal_chatbot/ipc_qa.json/* /content/my_legal_chatbot/
!mv /content/my_legal_chatbot/crpc_qa.json* /content/my_legal_chatbot/



mv: cannot stat '/content/my_legal_chatbot/constitution_qa.json/*': Not a directory
mv: cannot stat '/content/my_legal_chatbot/ipc_qa.json/*': Not a directory
mv: '/content/my_legal_chatbot/crpc_qa.json' and '/content/my_legal_chatbot/crpc_qa.json' are the same file


In [12]:
%cd /content/my_legal_chatbot


/content/my_legal_chatbot


In [14]:
!git commit -m "Add dataset files"


[master da22493] Add dataset files
 3 files changed, 58182 insertions(+)
 create mode 100644 constitution_qa.json
 create mode 100644 crpc_qa.json
 create mode 100644 ipc_qa.json


In [13]:
!git add .


In [ ]:
kajukhatri@123
#appointment.py

import sqlite3
import os

# Ensure database is created once at start
def create_database():
    conn = sqlite3.connect('appointments.db')
    cursor = conn.cursor()
    cursor.execute('''CREATE TABLE IF NOT EXISTS appointments (
                        id INTEGER PRIMARY KEY AUTOINCREMENT,
                        name TEXT,
                        phone TEXT UNIQUE,
                        date TEXT,
                        time TEXT)''')
    conn.commit()
    conn.close()

create_database()  # Only once at startup

def handle_appointment(message):
    data = message.split(",")
    if len(data) != 4:
        return "❌ Please provide: Name, Phone, Date (YYYY-MM-DD), Time (HH:MM AM/PM)"

    name, phone, date, time = map(str.strip, data)

    conn = sqlite3.connect('appointments.db')
    cursor = conn.cursor()

    try:
        cursor.execute("INSERT INTO appointments (name, phone, date, time) VALUES (?, ?, ?, ?)",
                       (name, phone, date, time))
        conn.commit()
        return f"✅ Appointment booked for {name} on {date} at {time}!"
    except sqlite3.IntegrityError:
        return "❌ Appointment with this phone number already exists!"

    finally:
        conn.close()

def view_appointment(phone):
    conn = sqlite3.connect('appointments.db')
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM appointments WHERE phone=?", (phone,))
    appointment = cursor.fetchone()
    conn.close()

    if appointment:
        return f"📅 Appointment for {appointment[1]} on {appointment[3]} at {appointment[4]}"
    else:
        return "❌ No appointment found for this phone number."

def delete_appointment(phone):
    conn = sqlite3.connect('appointments.db')
    cursor = conn.cursor()
    cursor.execute("DELETE FROM appointments WHERE phone=?", (phone,))
    conn.commit()
    conn.close()

    if cursor.rowcount > 0:
        return "✅ Appointment deleted successfully!"
    else:
        return "❌ No appointment found to delete."


In [ ]:
#legal_bot.py

import json
import torch
from sentence_transformers import SentenceTransformer
from transformers import GPT2LMHeadModel, GPT2Tokenizer

all_questions, all_answers = [], []
for filename in ['constitution_qa.json', 'crpc_qa.json', 'ipc_qa.json']:
    with open(filename, 'r', encoding='utf-8') as file:
        data = json.load(file)
        for item in data:
            all_questions.append(item['question'])
            all_answers.append(item['answer'])

embedder = SentenceTransformer('all-MiniLM-L6-v2')
question_embeddings = embedder.encode(all_questions, convert_to_tensor=True)

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

system_prompt = "You are a legal assistant. Answer the following legal or law-related question:"

legal_terms = ["article", "section", "law", "act", "IPC", "CrPC", "court", "bail", "petition", "FIR",
               "justice", "rights", "legal", "punishment", "constitution", "case", "crime", "order",
               "judge", "appeal", "writ", "contract", "arrest"]

def is_legal_question(question):
    return any(term.lower() in question.lower() for term in legal_terms)

def get_answer_if_found(user_question, threshold=0.75):
    user_embedding = embedder.encode(user_question, convert_to_tensor=True)
    similarities = torch.nn.functional.cosine_similarity(user_embedding, question_embeddings)

    best_idx = torch.argmax(similarities).item()
    best_similarity = similarities[best_idx].item()

    return (all_answers[best_idx], best_similarity) if best_similarity >= threshold else (None, best_similarity)

def generate_answer_with_gpt2(question):
    input_text = f"{system_prompt}\n\nQuestion: {question}\nAnswer:"
    input_ids = tokenizer.encode(input_text, return_tensors='pt')
    output_ids = model.generate(input_ids, max_length=200, pad_token_id=tokenizer.eos_token_id, temperature=0.7, top_p=0.9, do_sample=True)
    return tokenizer.decode(output_ids[0], skip_special_tokens=True).split("Answer:")[-1].strip()

def get_legal_answer(question):
    if not is_legal_question(question):
        return "❌ This chatbot only answers legal questions."
    answer, similarity = get_answer_if_found(question)
    if answer:
        return f"✅ Found Answer (Similarity: {similarity:.2f}):\n\n{answer}"
    else:
        return f"🤖 No direct match found. Generating AI answer...\n\n{generate_answer_with_gpt2(question)}"
